In [1]:
from pyspark import SparkConf, SparkContext
import math
from operator import add

In [2]:
sc.stop()

In [3]:
conf = SparkConf().setMaster("local").setAppName("HW3")
sc = SparkContext(conf=conf)
data = sc.textFile("data.txt")
c1 = sc.textFile("c1.txt")
c2 = sc.textFile("c2.txt")

In [4]:
iters = 20
k = 10

In [5]:
def mapper1(line):
    tmp = line.split(" ")
    dim = []
    for i in range (len(tmp)):
        dim.append(float(tmp[i]))
    return dim

In [6]:
c1 = c1.map(mapper1)
c2 = c2.map(mapper1)
data = data.map(mapper1)

In [7]:
cen1 = sc.broadcast(c1.collect())
cen2 = sc.broadcast(c2.collect())

In [8]:
def Euclidean1(x):
    min_dist = float("+inf")
    clust = 0
    for i in range (len(cen1.value)):
        t = 0
        for j in range (len(x)):
            t += ((x[j] - cen1.value[i][j])**2)
        t = math.sqrt(t)
        if t < min_dist:
            clust = i
            min_dist = t
    return(clust,min_dist**2,x)

In [9]:
def Euclidean2(x):
    min_dist = float("+inf")
    clust = 0
    for i in range (len(cen2.value)):
        t = 0
        for j in range (len(x)):
            t += ((x[j] - cen2.value[i][j])**2)
        t = math.sqrt(t)
        if t < min_dist:
            clust = i
            min_dist = t
    return(clust,min_dist**2,x)

In [10]:
def Manhattan1(x):
    min_dist = float("+inf")
    clust = 0
    for i in range (len(cen_1.value)):
        t = 0
        for j in range (len(x)):
            t += (abs(x[j]-cen_1.value[i][j]))
        if t < min_dist:
            clust = i
            min_dist = t
    return (clust,min_dist,x)

In [11]:
def Manhattan2(x):
    min_dist = float("+inf")
    clust = 0
    for i in range (len(cen_2.value)):
        t = 0
        for j in range (len(x)):
            t += (abs(x[j]-cen_2.value[i][j]))
        if t < min_dist:
            clust = i
            min_dist = t
    return (clust,min_dist,x)

In [12]:
def reduce1(x):
    t = []
    for j in range (len(x[1][0])): #num of features
        cnt = 0
        for i in range (len(x[1])): #num of same clust
            cnt += x[1][i][j]
        t.append(cnt/len(x[1]))
    return (t)

In [13]:
# Euclidean distance
for i in range (iters):
    c1_clust_E = data.map(Euclidean1)
    c2_clust_E = data.map(Euclidean2)
    c1_cost_E = c1_clust_E.map(lambda x:x[1]).reduce(add)
    c2_cost_E = c2_clust_E.map(lambda x:x[1]).reduce(add)
    print(i,round(c1_cost_E,2),round(c2_cost_E,2))
    new_cen1 = c1_clust_E.map(lambda x:(x[0],x[2])).groupByKey().mapValues(tuple).sortByKey().map(reduce1)
    new_cen2 = c2_clust_E.map(lambda x:(x[0],x[2])).groupByKey().mapValues(tuple).sortByKey().map(reduce1)
    cen1 = sc.broadcast(new_cen1.collect())
    cen2 = sc.broadcast(new_cen2.collect())

0 623660345.31 438747790.03
1 509862908.3 249803933.63
2 485480681.87 194494814.41
3 463997011.69 169804841.45
4 460969266.57 156295748.81
5 460537847.98 149094208.11
6 460313099.65 142508531.62
7 460003523.89 132303869.41
8 459570539.32 117170969.84
9 459021103.34 108547377.18
10 458490656.19 102237203.32
11 457944232.59 98278015.75
12 457558005.2 95630226.12
13 457290136.35 93793314.05
14 457050555.06 92377131.97
15 456892235.62 91541606.25
16 456703630.74 91045573.83
17 456404203.02 90752240.1
18 456177800.54 90470170.18
19 455986871.03 90216416.18


In [14]:
c1_clust_E = data.map(Euclidean1)
c2_clust_E = data.map(Euclidean2)
c1_cost_E = c1_clust_E.map(lambda x:x[1]).reduce(add)
c2_cost_E = c2_clust_E.map(lambda x:x[1]).reduce(add)
print("20 ",round(c1_cost_E,2) ,round(c2_cost_E,2))

20  455729268.36 90162390.91


In [15]:
cen_1 = sc.broadcast(c1.collect())
cen_2 = sc.broadcast(c2.collect())

In [16]:
# Manhattan distance
for i in range (iters):
    c1_clust_M = data.map(Manhattan1)
    c2_clust_M = data.map(Manhattan2)
    c1_cost_M = c1_clust_M.map(lambda x:x[1]).reduce(add)
    c2_cost_M = c2_clust_M.map(lambda x:x[1]).reduce(add)
    print(i,round(c1_cost_M,2),round(c2_cost_M,2))
    new_cen1 = c1_clust_M.map(lambda x:(x[0],x[2])).groupByKey().mapValues(tuple).sortByKey().map(reduce1)
    new_cen2 = c2_clust_M.map(lambda x:(x[0],x[2])).groupByKey().mapValues(tuple).sortByKey().map(reduce1)
    cen_1 = sc.broadcast(new_cen1.collect())
    cen_2 = sc.broadcast(new_cen2.collect())

0 550117.14 1433739.31
1 464869.28 1084488.78
2 470897.38 973431.71
3 483914.41 895934.59
4 489216.07 865128.34
5 487629.67 845846.65
6 483711.92 827219.58
7 475330.77 803590.35
8 474871.24 756039.52
9 457232.92 717332.9
10 447494.39 694587.93
11 450915.01 684444.5
12 451250.37 674574.75
13 451974.6 667409.47
14 451570.36 663556.63
15 452739.01 660162.78
16 453082.73 656041.32
17 450583.67 653036.75
18 450368.75 651112.43
19 449011.36 649689.01


In [17]:
c1_clust_M = data.map(Manhattan1)
c2_clust_M = data.map(Manhattan2)
c1_cost_M = c1_clust_M.map(lambda x:x[1]).reduce(add)
c2_cost_M = c2_clust_M.map(lambda x:x[1]).reduce(add)
print("20 ",round(c1_cost_M,2),round(c2_cost_M,2))

20  446770.42 646481.16


# Distance between centroid for each version

In [26]:
# Euclidean strategy+ c1 + Euclidean distance computing
for i in range (len(cen1.value)): # num of centroids
    for j in range (i+1,len(cen1.value)):
        dist = 0
        for k in range (len(cen1.value[i])): # num of features
            dist += ((cen1.value[i][k]-cen1.value[j][k])**2)
        dist = math.sqrt(dist)
        print(i,j,round(dist,2))

0 1 692.16
0 2 3490.26
0 3 205.75
0 4 346.72
0 5 512.61
0 6 444.73
0 7 566.2
0 8 1282.77
0 9 307.67
1 2 2798.8
1 3 897.66
1 4 1038.83
1 5 1204.08
1 6 1136.33
1 7 1257.45
1 8 669.89
1 9 412.08
2 3 3695.11
2 4 3836.91
2 5 4002.69
2 6 3934.87
2 7 4056.14
2 8 2294.58
2 9 3195.92
3 4 142.44
3 5 309.51
3 6 241.73
3 7 363.26
3 8 1474.95
3 9 504.63
4 5 167.15
4 6 99.55
4 7 220.9
4 8 1615.85
4 9 646.93
5 6 67.91
5 7 53.79
5 8 1782.2
5 9 814.08
6 7 121.63
6 8 1715.25
6 9 746.34
7 8 1835.64
7 9 867.82
8 9 975.32


In [27]:
# Euclidean strategy+ c1 + Manhattan distance computing
for i in range (len(cen1.value)): # num of centroids
    for j in range (i+1,len(cen1.value)):
        dist = 0
        for k in range (len(cen1.value[i])): # num of features
            dist += (abs(cen1.value[i][k]-cen1.value[j][k]))
        print(i,j,round(dist,2))

0 1 728.92
0 2 3797.9
0 3 212.18
0 4 374.89
0 5 577.4
0 6 499.16
0 7 645.77
0 8 1731.06
0 9 406.7
1 2 3072.89
1 3 935.89
1 4 1100.83
1 5 1303.9
1 6 1225.35
1 7 1372.09
1 8 1005.29
1 9 490.93
2 3 4001.04
2 4 4170.3
2 5 4372.79
2 6 4294.95
2 7 4440.72
2 8 2513.42
2 9 3396.42
3 4 171.37
3 5 375.25
3 6 296.25
3 7 443.5
3 8 1934.09
3 9 609.75
4 5 204.52
4 6 125.6
4 7 272.93
4 8 2102.86
4 9 779.4
5 6 79.4
5 7 69.59
5 8 2306.38
5 9 983.02
6 7 147.87
6 8 2227.56
6 9 904.37
7 8 2374.55
7 9 1050.92
8 9 1327.58


In [28]:
# Euclidean strategy+ c2 + Euclidean distance computing
for i in range (len(cen2.value)):
    for j in range (i+1,len(cen2.value)):
        dist = 0
        for k in range (len(cen2.value[i])):
            dist += ((cen2.value[i][k]-cen2.value[j][k])**2)
        dist = math.sqrt(dist)
        print(i,j,round(dist,2))

0 1 15760.12
0 2 14110.83
0 3 9045.32
0 4 5567.68
0 5 1924.62
0 6 1100.86
0 7 402.89
0 8 2105.44
0 9 3169.0
1 2 11524.51
1 3 6743.88
1 4 10192.53
1 5 14455.12
1 6 14682.45
1 7 15362.42
1 8 13674.71
1 9 12597.04
2 3 9545.88
2 4 10883.38
2 5 12233.96
2 6 13208.0
2 7 13786.48
2 8 12508.96
2 9 11938.38
3 4 3494.22
3 5 7718.22
3 6 7957.78
3 7 8644.81
3 8 6947.82
3 9 5876.33
4 5 4404.56
4 6 4492.46
4 7 5169.94
4 8 3488.16
4 9 2407.92
5 6 1182.86
5 7 1615.79
5 8 1313.33
5 9 2153.77
6 7 698.49
6 8 1010.2
6 9 2085.46
7 8 1702.79
7 9 2768.61
8 9 1080.53


In [29]:
# Euclidean strategy+ c2 + Manhattan distance computing
for i in range (len(cen2.value)):
    for j in range (i+1,len(cen2.value)):
        dist = 0
        for k in range (len(cen2.value[i])):
            dist += (abs(cen2.value[i][k]-cen2.value[j][k]))
        print(i,j,round(dist,2))

0 1 15772.61
0 2 20215.65
0 3 9533.17
0 4 5604.2
0 5 3088.05
0 6 1311.04
0 7 471.27
0 8 2369.41
0 9 3349.66
1 2 16003.5
1 3 7219.2
1 4 10221.03
1 5 16105.35
1 6 14909.17
1 7 15434.46
1 8 13950.58
1 9 12776.88
2 3 10690.48
2 4 14613.55
2 5 17509.9
2 6 18912.61
2 7 19748.94
2 8 17851.81
2 9 16873.24
3 4 3935.29
3 5 8896.39
3 6 8228.36
3 7 9065.4
3 8 7168.73
3 9 6190.68
4 5 5893.07
4 6 4696.98
4 7 5221.25
4 8 3737.71
4 9 2564.17
5 6 1781.82
5 7 2619.81
5 8 2162.8
5 9 3337.75
6 7 840.72
6 8 1068.94
6 9 2137.79
7 8 1901.21
7 9 2883.73
8 9 1176.45


In [30]:
# Manhattan strategy+ c1 + Euclidean distance computing
for i in range (len(cen_1.value)):
    for j in range (i+1,len(cen_1.value)):
        dist = 0
        for k in range (len(cen_1.value[i])):
            dist += ((cen_1.value[i][k]-cen_1.value[j][k])**2)
        dist = math.sqrt(dist)
        print(i,j,round(dist,2))

0 1 2219.18
0 2 9948.04
0 3 528.7
0 4 413.37
0 5 827.72
0 6 681.03
0 7 917.13
0 8 832.15
0 9 729.06
1 2 7767.95
1 3 2734.05
1 4 2628.49
1 5 3044.48
1 6 2898.71
1 7 3133.46
1 8 1812.45
1 9 1491.36
2 3 10433.06
2 4 10361.37
2 5 10773.53
2 6 10626.49
2 7 10862.97
2 8 9340.28
2 9 9236.84
3 4 221.37
3 5 375.16
3 6 249.38
3 7 457.26
3 8 1156.58
3 9 1251.16
4 5 415.99
4 6 270.75
4 7 505.07
4 8 1171.96
4 9 1137.14
5 6 147.05
5 7 89.49
5 8 1529.46
5 9 1553.12
6 7 236.51
6 8 1391.55
6 9 1407.4
7 8 1613.56
7 9 1642.13
8 9 709.41


In [31]:
# Manhattan strategy+ c1 + Manhattan distance computing
for i in range (len(cen_1.value)):
    for j in range (i+1,len(cen_1.value)):
        dist = 0
        for k in range (len(cen_1.value[i])):
            dist += (abs(cen_1.value[i][k]-cen_1.value[j][k]))
        print(i,j,round(dist,2))

0 1 2341.02
0 2 11929.3
0 3 651.19
0 4 496.33
0 5 947.74
0 6 770.74
0 7 1056.8
0 8 1260.51
0 9 737.71
1 2 9597.44
1 3 2778.95
1 4 2830.14
1 5 3280.36
1 6 3104.29
1 7 3388.98
1 8 2380.46
1 9 1605.27
2 3 12323.29
2 4 12421.26
2 5 12871.48
2 6 12695.55
2 7 12979.13
2 8 10775.94
2 9 11196.79
3 4 335.95
3 5 558.47
3 6 382.46
3 7 667.53
3 8 1653.83
3 9 1379.17
4 5 452.86
4 6 276.33
4 7 561.85
4 8 1755.11
4 9 1226.66
5 6 177.59
5 7 110.22
5 8 2205.31
5 9 1677.67
6 7 287.43
6 8 2028.9
6 9 1500.99
7 8 2314.67
7 9 1786.81
8 9 1006.37


In [32]:
# Manhattan strategy+ c2 + Euclidean distance computing
for i in range (len(cen_2.value)):
    for j in range (i+1,len(cen_2.value)):
        dist = 0
        for k in range (len(cen_2.value[i])):
            dist += ((cen_2.value[i][k]-cen_2.value[j][k])**2)
        dist = math.sqrt(dist)
        print(i,j,round(dist,2))

0 1 15747.23
0 2 14100.14
0 3 9032.33
0 4 5554.79
0 5 2006.7
0 6 1338.16
0 7 514.63
0 8 1571.24
0 9 3022.66
1 2 11524.51
1 3 6743.88
1 4 10192.53
1 5 14474.55
1 6 14412.06
1 7 15239.88
1 8 14328.23
1 9 12731.4
2 3 9545.88
2 4 10883.38
2 5 12167.79
2 6 13125.35
2 7 13684.61
2 8 12643.99
2 9 12006.39
3 4 3494.22
3 5 7742.63
3 6 7694.28
3 7 8521.2
3 8 7588.4
3 9 6009.82
4 5 4452.97
4 6 4219.76
4 7 5047.52
4 8 4167.64
4 9 2542.57
5 6 1405.11
5 7 1637.73
5 8 910.99
5 9 2124.26
6 7 827.84
6 8 566.55
6 9 1684.52
7 8 1081.38
7 9 2511.46
8 9 1649.39


In [33]:
# Manhattan strategy + c2 + Manhattan distance computing
for i in range (len(cen_2.value)):
    for j in range (i+1,len(cen_2.value)):
        dist = 0
        for k in range (len(cen_2.value[i])):
            dist += (abs(cen_2.value[i][k]-cen_2.value[j][k]))
        print(i,j,round(dist,2))

0 1 15757.69
0 2 20200.26
0 3 9517.67
0 4 5588.85
0 5 3281.49
0 6 1430.21
0 7 602.95
0 8 2102.55
0 9 3211.46
1 2 16003.5
1 3 7219.2
1 4 10221.03
1 5 16325.27
1 6 14506.49
1 7 15335.96
1 8 14980.06
1 9 12922.93
2 3 10690.48
2 4 14613.55
2 5 17521.52
2 6 18775.12
2 7 19602.26
2 8 18111.89
2 9 16995.13
3 4 3935.29
3 5 9116.02
3 6 8090.51
3 7 8918.81
3 8 7771.22
3 9 6312.53
4 5 6110.83
4 6 4293.5
4 7 5123.07
4 8 4768.92
4 9 2710.06
5 6 1855.58
5 7 2682.57
5 8 1358.8
5 9 3413.04
6 7 833.43
6 8 674.83
6 9 1784.51
7 8 1500.82
7 9 2614.0
8 9 2062.25
